### Import all relevant dependencies

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from pathlib import Path
import yaml
from src.minimizer.minimizer_library.differential_evolution_parallel import DifferentialEvolutionParallel
from src.statistical_models.statistical_model_library.fcs_gaussian_noise_model import FCSGaussianNoiseModel
from src.visualization.plotting_functions import *
from src.math_utils.derivatives.numeric_derivative_calculator import NumericDerivativeCalculator
from src.model.hahn_stack_model import HahnStackModel
from src.math_utils.scaler.hahn_parameter_scaler import HahnParameterScaler
from src.model.parameter_set.hahn_parameter_set import HahnParameterSet
from src.utils.experiment_serialization import import_experiment_results

### Define the Experiment Metadata
- bounds for the free parameters
- operating conditions
- experiment repetitions
- true parameter set
- Experiment variance
- current values


Naming:
unscaled/ rescaled: actual physical values
scaled: scaled value between 0 & 1

In [2]:
path = "../data/config/reduced_example_config_workflow.yaml"

with open(path, "r") as f:
    config = yaml.safe_load(f)

number_designs = config["number_designs"] #Amount of LH Designs
n_rep = config["n_rep"] #Amount of experiment repetitions
n_current_values = config["n_current_values"] #Amount of individual current values
sigma = config["sigma"] #experiment variance (10mV variance in repeated experiments)

# lower and upper bounds for operating conditions
upper_bounds_operating_conditions = np.array(config["upper_bounds_operating_conditions"])
lower_bounds_operating_conditions = np.array(config["lower_bounds_operating_conditions"])

I_S_array = np.linspace(1, 600, n_current_values) # initialize applicable current array request

# select parameter values and names to be analyzed
unscaled_theta_true = np.array([list(HahnParameterSet().free_parameters.values())[i] for i in [1, 2, 4]])
names_theta = [list(HahnParameterSet().free_parameters.keys())[i] for i in [1, 2, 4]]

# initialize lower and upper bounds for free parameter values
unscaled_upper_bounds_free_params = np.array(config["unscaled_upper_bounds_free_params"], dtype=float)
unscaled_lower_bounds_free_params = np.array(config["unscaled_lower_bounds_free_params"], dtype=float)

print(names_theta, unscaled_lower_bounds_free_params, unscaled_theta_true, unscaled_upper_bounds_free_params)

['j_0_ref', 'r_el', 'D_GDL_ref'] [1.e+01 1.e-06 1.e-06] [2.1308e+03 4.2738e-06 8.6266e-06] [5.e+03 1.e-03 1.e-05]


### Define Scaler and scale parameter values as well as  bounds of operating conditions and parameters
Scalers are saved in variable "scaler" and handed over to stack model.

In [3]:
scaler = HahnParameterScaler() # define scaler

# Stack bounds of free parameters
free_parameter_bounds = np.vstack([
    unscaled_lower_bounds_free_params,
    unscaled_upper_bounds_free_params
]).T

# Stack operating condition bounds (rows = condition, columns = [min, max])
operating_condition_bounds = np.vstack([
    lower_bounds_operating_conditions,
    upper_bounds_operating_conditions
]).T

# Determine current range for scaling
current_bounds = np.array([[I_S_array.min(), I_S_array.max()]])

scaled_theta_true = scaler.scale_theta(unscaled_theta_true, free_parameter_bounds)

# scale bounds of operating conditions to hand over for Experimental designs incl. LHC
scaled_upper_bounds = scaler.scale_params(upper_bounds_operating_conditions, operating_condition_bounds)
scaled_lower_bounds = scaler.scale_params(lower_bounds_operating_conditions, operating_condition_bounds)

# scale bounds of free parameters for initializing Model
scaled_lower_bounds_theta, _ = scaler.scale(unscaled_lower_bounds_free_params, free_parameter_bounds)
scaled_upper_bounds_theta, _ = scaler.scale(unscaled_upper_bounds_free_params, free_parameter_bounds)

crlb_factor = unscaled_upper_bounds_free_params-unscaled_lower_bounds_free_params # offset for reformating the CRLB
unit_factors = np.array([1e-4, 1e4, 1e4]) #[1e-3, 1e-4, 1e4, 1e4, 1e4, 1] factors to reformat the units from SI to applicable industry standard units
# Print scaling results of scaled true parameters
print("Scalers theta:", scaler.theta_scalers)
print("Scaled theta:", scaled_theta_true)
print("Rescaled theta:", unscaled_theta_true)

Scalers theta: [MinMaxScaler(), MinMaxScaler(), MinMaxScaler()]
Scaled theta: [0.42501002 0.00327708 0.8474    ]
Rescaled theta: [2.1308e+03 4.2738e-06 8.6266e-06]


### Define the parametric function including handover of bounds

In [4]:
hahn_fc_model = HahnStackModel(parameter_set=HahnParameterSet(free_parameters=names_theta)) #Instanciate the generic Hahn Model
calculator = NumericDerivativeCalculator(hahn_fc_model, scaler) #Instanciate the derivation calculator function

#Initiallize statistical model function
statistical_model = FCSGaussianNoiseModel(model_function=hahn_fc_model,
                                          der_function=calculator,
                                          lower_bounds_x=scaled_lower_bounds,
                                          upper_bounds_x=scaled_upper_bounds,
                                          lower_bounds_theta=scaled_lower_bounds_theta,
                                          upper_bounds_theta=scaled_upper_bounds_theta,
                                          sigma=sigma,
                                          scaler = scaler,)

#Initialize blackbox function returning noised experiment results
def blackbox_model(x):
    return statistical_model.random(theta=scaled_theta_true, x=x)

### Import LH Designs from 04A - calculate base designs

In [5]:
df_exp_input = import_experiment_results(".." / Path("data") / "experimental_designs" / "lhc"/"lhc.csv")

# prepare LHC data for further analysis
x_designs = df_exp_input[['Pressure', 'Temperature', 'Stoichiometry', 'Current']].to_numpy()
y_designs = df_exp_input[['Voltage']].to_numpy()

### Initialize minimizer and hyperparameters for additional experiments

In [6]:
iterations = 100 # maximum iterations of minimizer function

minimizer = DifferentialEvolutionParallel(maxiter=iterations, n_workers=-1, display=False)

In [7]:
# Estimate thetas, estimated are for every experiment setup with all current values one theta, so n_theta_estimations = n_rep
scaled_estimated_thetas = statistical_model.estimate_repeated_thetas(
    x0=x_designs, y=y_designs, n=n_rep, minimizer=minimizer)

Estimating thetas: 100%|██████████| 100/100 [02:50<00:00,  1.71s/it]


In [8]:
# save estimated thetas
root = Path.cwd().parent   # this will be oed_fuel_cell_model/
data_path = root / "data" / "estimated_parameters" / "lhc"

out_file = data_path / "estimated_thetas.csv"
np.savetxt(out_file, scaled_estimated_thetas, delimiter=",")